In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
df = pd.read_csv('stats_tourisme.csv')
df.head()
sns.set_style('darkgrid')

La fonction suivante va permettre d'ajouter les noms de chaque pays à côté des points correspondants sur les graphes que nous allons tracer :

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']))

## Disponibilité du voyage aérien

Le tourisme international s'est beaucoup développé grâce à la démocratisation de l'avion. La plupart des touristes utilisent ce moyen de transport pour traverser les frontières, qu'elles soient terrestres ou marines.

On va commencer par regarder s'il y a un lien entre le nombre d'habitants et le nombre de voyageurs aériens pour tous les pays.

In [ ]:
masque = (df['Touristes annuels'].notna()) & (df['Nombre de passagers aériens'].notna())
touristes_par_avion = df.loc[masque, ['Pays', 'Touristes annuels', 'Nombre de passagers aériens', 'PIB / habitant']]
graphique = sns.relplot(data=touristes_par_avion, x='Population', y='Nombre de passagers aériens')
graphique.set(xscale = 'log', title = "Voyageurs aériens en fonction de la population");

label_point(touristes_par_avion['Population'],touristes_par_avion['Nombre de passagers aériens'], touristes_par_avion['Pays'], plt.gca())

Cela semble être le cas puisqu'on obtient une courbe qui ressemble à une exponentielle, traduisant une relation linéaire entre nos deux paramètres.

L'avion est un moyen de transport qui nécessite d'importantes infrastructures. On peut donc imaginer que ce soit en priorité les pays les plus riches qui utilisent l'avion.

In [ ]:

touristes_par_avion['Voyageurs aériens / population'] = touristes_par_avion['Nombre de passagers aériens'] / touristes_par_avion['Population']

#On élimine les pays avec trop de voyageurs aériens pour pouvoir visualiser correctement les autres
exclusion_pays = touristes_par_avion[(touristes_par_avion['Pays'] != 'Ireland') & (touristes_par_avion['Pays'] != 'Luxembourg') & (touristes_par_avion['Pays'] != 'Cyprus') & (touristes_par_avion['Pays'] != 'Iceland')]

graphique = sns.regplot(data=exclusion_pays, x='PIB / habitant', y='Voyageurs aériens / population')
graphique.set(title = "Voyageurs aériens en fonction du PIB et la population");

label_point(touristes_par_avion['PIB / habitant'],touristes_par_avion['Voyageurs aériens / population'], exclusion_islande['Pays'], plt.gca())


Lorsque le PIB par habitant d'un pays augmente, les habitants ont tendance à plus souvent prendre l'avion ;  et ce quel que soit le continent. L'avion est donc une caractéristique des pays les plus riches, mais est-ce que cela se reflète sur le tourisme ?

Plus d'avions vers un pays devrait vouloir dire plus de touristes ?

In [ ]:
touristes_par_avion['Voyageurs aériens / nombre de touristes'] = touristes_par_avion['Nombre de passagers aériens'] / touristes_par_avion['Touristes annuels']

graphique = sns.lmplot(data=touristes_par_avion, x='PIB / habitant', y='Voyageurs aériens / nombre de touristes')
graphique.set(title = "Voyageurs aériens en fonction du PIB et du nombre de touristes");

label_point(touristes_par_avion['PIB / habitant'],touristes_par_avion['Voyageurs aériens / nombre de touristes'], touristes_par_avion['Pays'], plt.gca())


Ici encore on obtient une fonction croissante du PIB par habitant.

Néanmoins, pour la majorité des pays, le nombre de voyages aériens est bien supérieur au nombre de touristes : il y a au moins deux fois plus de voyages en avion que de visiteurs annuels du pays.
On en déduit que le tourisme ne concerne finalement qu'une faible part du trafic aérien. L'essentiel de ce domaine est probablement liée au business.

Il est donc difficile de tirer des conclusions sur le tourisme à partir de résultats sur les voyages en avion.

## Tourisme "local"

Dans le cadre de la crise liée à la pandémie de Covid-19, le secteur du tourisme a été particulièrement affecté par la suppression des trajets internationaux et/ou de longue distance. Pour survivre économiquement, les acteurs de ce secteur ont du compter sur un tourisme plus local, et notamment sur leurs propres compatriotes.

En effet, au lieu de voyager à l'étranger, une partie des touristes a choisi de se déplacer moins loin, et souvent sans changer de pays. 

Ainsi, en temps de crise, le tourisme intranational pourrait devenir la solution la plus efficace - mais tous les pays ne sont pas égaux dans ce domaine.